# Widzenie komputerowe - Laboratoria nr 8

## Opis laboratoriów

Segmentacja obrazu bazująca na:
- progowaniu
- analizie skupień,
- wykrywaniu cech obrazu (np. krawędzi),
- podziale i rozroście,



## Funkcje pomocnicze

### Niezbędne biblioteki


- OpenCV - główna biblioteka do przetwarzania obrazów, zawierająca operacje i algorytmy na obrazach,
- matplotlib, pillow, pandas (głównie do wizualizacji),
- numpy - przechowywanie i operacje na danych,
- dodatkowo: json, os, skimage

In [ ]:
# to have popular descriptors like SIFT
!pip install -U opencv-python
!pip install -U opencv-contrib-python

In [ ]:
# import niezbędnych bibliotek
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import PIL
from pandas import DataFrame
import pandas as pd
from IPython.display import display, HTML
from skimage.exposure import rescale_intensity
import json
import os
from itertools import product
import itertools
import random
from sklearn.mixture import GaussianMixture
import plotly.graph_objects as go
import sys

pd.options.display.html.border = 0
pd.options.display.float_format = '{:,.2f}'.format

### Zbiory danych

* obraz Lenna (dostępny pod [linkiem](http://www.lenna.org/)) - jeden z najbardziej popularnych obrazów wykorzystywanych historycznie do kompresji i przetwarzania obrazów,
* clevr - obraz prezentacyjny pochodzący ze zbioru danych CLEVR zajmującego się problemem Visual Query Answering,
* graf - przykładowy obraz grafiti z repozytorium OpenCV,
* sudoku - przykładowy obraz sudoku z repozytorium OpenCV,
* skittles - obraz skittlesów pochodzący z wikimedia.org

In [ ]:
# pobranie niezbędnych bibliotek
!wget -O lena_std.tif http://www.lenna.org/lena_std.tif
!wget -O clevr.jpg https://cs.stanford.edu/people/jcjohns/clevr/teaser.jpg
!wget -O graf.png https://github.com/opencv/opencv/raw/master/samples/data/graf1.png
!wget -O sudoku.png https://raw.githubusercontent.com/opencv/opencv/master/samples/data/sudoku.png
!wget -O skittles.jpg https://upload.wikimedia.org/wikipedia/commons/c/ca/Skittles-Louisiana-2003.jpg

### Wizualizacja

- imshow() - wizualizacja obrazów BGR/BGRA,
- h() - wizualizacja tabeli w postaci numerycznej,
- h_color() - wizualizacja tabeli w postaci kolorystycznej,
- h_grid() - wizualizacja rastra ARC,

In [ ]:
def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(PIL.Image.fromarray(a))

In [ ]:
css = """
<style type="text/css">
  table, td, table.dataframe, table.dataframe td {
    border: 1px solid black;    //border: double;
    border-collapse: collapse;
    border-style: solid;
    border-spacing: 0px;
    background-color: rgb(250,250,250);
    width: 18px;
    height: 18px;
    text-align: center;
    transform: scale(1.0);
    margin: 2px;
    }
</style>
"""

def h(s):
  return display(HTML(css + DataFrame(s).to_html(header=False, index=False)))

In [ ]:
def h_color(a, cmap='gray', scale=2):
  s = [a.shape[0] * scale, a.shape[1] * scale]
  plt.figure(figsize=s)
  plt.tick_params(
    axis='both', which='both',
    bottom=False, top=False,
    labelbottom=False, labelleft=False, left=False, right=False
  )
  plt.imshow(a, cmap=cmap)

In [ ]:
cmap = ListedColormap([
    'black', 'tomato', 'chocolate', 'darkorange',
    'gold', 'olive', 'green', 'deepskyblue',
    'blueviolet', 'hotpink'
])

def h_grid(grid, scale=1):
  h_color(grid, cmap, scale)

In [ ]:
def pix_show(pixels, skip_each=1, height=800, width=800, colors=None):
  pixels = pixels[::skip_each]
  if colors is None: colors = pixels[:, ::-1]
  else: colors = colors[::skip_each]
  b, g, r = pixels[:, 0], pixels[:, 1], pixels[:, 2]
  fig = go.Figure(data=[
    go.Scatter3d(x=b, y=g, z=r, mode='markers', marker={
        'size': 2,
        'color': colors,
        'opacity': 0.7
    })
  ], layout_xaxis_range=[0, 1], layout_yaxis_range=[0, 1])
  scene = {
      'xaxis': dict(title='Blue'),
      'yaxis': dict(title='Green'),
      'zaxis': dict(title='Red')
  }
  fig.update_layout(autosize=False, height=height, width=width, scene=scene, showlegend=True)
  fig.show()

# Segmentacja obrazu

## Segmentacja poprzez progowanie

Najprostszym sposobem aby posegmentować obraz jest wykonanie progowania na intensywnościach pikseli. Operacja progowania polega na zastąpieniu pewną stałą wartością wszystkich intensywności powyżej pewnego progu, oraz inną wartością poniżej tego progu.

Istnieje również segmentacja z wieloma progami, co zostało zaprezentowane już na pierwszych zajęciach.

In [ ]:
lena = cv2.imread('./lena_std.tif', cv2.IMREAD_COLOR)
imshow(lena)

In [ ]:
lena_gray = cv2.cvtColor(lena, cv2.COLOR_BGR2GRAY)
imshow(lena_gray)

In [ ]:
lut = np.array([255] * 100 + [0] * 100 + [255] * 56)
lena_lut = cv2.LUT(lena_gray, lut)
imshow(lena_lut)

Biblioteka OpenCV zawiera gotową implementację również innych podejść prostego progowania obrazu. Aby wykonać operację progowania w OpenCV należy wywołać funkcję **threshold()**, która pobiera obraz, wartość progu, wartość maksymalną oraz metodę progowania, która powinna zostać zastosowana.

Wśród dostępnych metod progowania są dostępne m.in.:
- binary - dla pikseli o intensywnościach poniżej progu przypisana jest wartość **0**, dla pozostałych **wartość maksymalna**,
- binary inverted - dla pikseli o intensywnościach poniżej progu przypisana jest wartość **maksymalna**, dla pozostałych **0**,
- truncate - dla pikseli o intensywnościach powyżej progu przypisana jest wartość **progu**, pozostałe bez zmian,
- to zero - dla pikseli o intensywnościach poniżej progu przypisana jest wartość **0**, pozostałe bez zmian,
- to zero inverted - dla pikseli o intensywnościach powyżej progu przypisana jest wartość **0**, pozostałe bez zmian,

In [ ]:
_, lena_bin = cv2.threshold(lena_gray, 127, 255, cv2.THRESH_BINARY)
_, lena_bin_inv = cv2.threshold(lena_gray, 127, 255, cv2.THRESH_BINARY_INV)
_, lena_trunc = cv2.threshold(lena_gray, 127, 255, cv2.THRESH_TRUNC)
_, lena_tozero = cv2.threshold(lena_gray, 127, 255, cv2.THRESH_TOZERO)
_, lena_tozero_inv = cv2.threshold(lena_gray, 127, 255, cv2.THRESH_TOZERO_INV)

imshow(np.concatenate([lena_gray, lena_bin, lena_bin_inv], 1))
imshow(np.concatenate([lena_trunc, lena_tozero, lena_tozero_inv], 1))

Wśród metod progowania można wyróżnić również metody **adaptacyjne.** Sa to metody progowania, które dostosowują wartość progu w zależności od zawartości obrazu.

Metody progowania adaptacyjnego działają często bardzo dobrze gdy obraz wejściowy podzieli się na mniejsze obszary a wartość progowania zostanie dla każdego obszaru dostosowana osobno. **Motywacją stojącą za takim mechanizmem jest fakt, że na rzeczywistych obrazach oświetlenie (jak i ostrość, balans, itp.) jest nierównomierny.**

In [ ]:
lena_ad_mean = cv2.adaptiveThreshold(lena_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 101, 2)
lena_ad_gauss = cv2.adaptiveThreshold(lena_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 2)

imshow(np.concatenate([lena_ad_mean, lena_ad_gauss], 1))

### Przykład sudoku

In [ ]:
sudoku = cv2.imread('./sudoku.png', cv2.IMREAD_GRAYSCALE)
imshow(sudoku)
print(sudoku.shape)

In [ ]:
_, sudoku_bin = cv2.threshold(sudoku, 70, 255, cv2.THRESH_BINARY)
imshow(sudoku_bin)

In [ ]:
lut = np.array([0] * 50 + [255] * 80 + [0] * 126)
sudoku_lut = cv2.LUT(sudoku, lut)
imshow(sudoku_lut)

In [ ]:
sudoku_ad_mean = cv2.adaptiveThreshold(sudoku, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 101, 2)
sudoku_ad_gauss = cv2.adaptiveThreshold(sudoku, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 2)

imshow(np.concatenate([sudoku_ad_mean, sudoku_ad_gauss], 1))

### OTSU

OTSU to algorytm który adaptacyjnie dobiera taką wartość progu, aby intensywności obu nowych klas (zbinaryzowanych) miały wewnętrzenie najmniejszą wariancję intensywności pikseli (co jest równoznaczne maksymalizacji wariancji międzyklasowej).

Minimalizowaną wartością przez OTSU jest:
$$\sigma^2_w(t) = Q_1\sigma^2_1 + Q_2\sigma^2_2$$
gdzie:
- $Q_i$ - prawdopodobieństwo przynależności piksela do klasy i-tej (jest to wartość dystrybuanty, co wynika ze wzoru),
$$Q_0 = P(f(x,y) < t_0)$$
$$Q_1 = 1 - P(f(x,y) < t_0) $$
- $\sigma^2_i$ - wariancja wewnątrz klasy i-tej.

Aby znaleźć najmniejszą wartość wyrażenia można policzyć wszystkie wartości wyrażenia (od 0 do 255).

Pierwszym krokiem będzie obliczenie prawdopodobieństwa wystąpienia piksela o danej intensywności (od 0 do 255). Aby takie znaleźć, wystarczy wyznaczyć histogram oraz go znormalizować. Dodatkowo, policzona zostanie dystrybuanta na potrzeby obliczenia średniej i wariancji **warunkowej**.

In [ ]:
def probs(img):
  h = cv2.calcHist([img], [0], None, [256], [0, 256])
  pdf = h.ravel() / h.sum()
  cdf = np.cumsum(pdf)
  return pdf, cdf

pdf, cdf = probs(sudoku)

plt.plot(pdf)
plt.show()

plt.plot(cdf)
plt.show()

Pojedyncza iteracja (dla pewnego ustalonego progu) algorytmu OTSU polega na podziale rozkładu prawdopodobieństwa na prawdopodobieństwa wystąpienia pikseli jednej i drugiej klasy. Do wyznaczenia wartości średniej niezbędne jest skorzystanie ze wzoru na **warunkową wartość oczekiwaną** (ponieważ liczymy średnią z pikseli **pod warunkiem**, że zachodzi pewna klasa, lub innymi słowy: ''wartość oczekiwaną dla danej klasy'').

Następnie, do wyliczenia wariancji (waunkowych !) korzystamy z wcześniej obliczonych warunkowych wartości oczekiwanych.

In [ ]:
def cond_mean(i, p, q):
  # conditional expectation: E(X | A) = sum(x * P(x n A)) / P(A)
  return (i * p / q).sum()

def _otsu(i, pdf, cdf, epsilon=1e-6):
  i1, i2 = np.hsplit(np.arange(256), [i])
  p1, p2 = np.hsplit(pdf, [i])
  q1, q2 = cdf[i], cdf[-1] - cdf[i]

  if q1 < epsilon or q2 < epsilon:
      return None

  m1, m2 = cond_mean(i1, p1, q1), cond_mean(i2, p2, q2)
  s1, s2 = cond_mean((i1 - m1) ** 2, p1, q1), cond_mean((i2 - m2) ** 2, p2, q2)

  return s1 * q1 + s2 * q2

W najbardziej podstawowej wersji algorytm OTSU, jeśli to możliwe, polega na przeiterowaniu wszystkich możliwych podziałów (threshold od 1 do 254) i wybraniu takiego, dla którego wcześniej przedstawiona funkcja celu zwraca najmniejszą wartość.

In [ ]:
def otsu(img):
  pdf, cdf = probs(img)
  v_min = None
  threshold = 0

  for i in range(1, 256):
    v = _otsu(i, pdf, cdf)
    if v is not None and (v_min is None or v < v_min):
      v_min = v
      threshold = i

  _, img_otsu = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)

  return threshold, img_otsu

W celu porównania wyników wykonany został algorytm progowania OTSU korzystając z biblioteki OpenCV (wystarczy dodać flag THRESH_OTSU do operacji progowania).

In [ ]:
th_self, sudoku_otsu_self = otsu(sudoku)
th_auto, sudoku_otsu_auto = cv2.threshold(sudoku, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print('Znaleziona wartość progowania algorytmem OTSU (własna implementacja):', th_self)
print('Znaleziona wartość progowania algorytmem OTSU (OpenCV):', th_auto)

print('\nOTSU (własna implementacja):')
imshow(sudoku_otsu_self)
print('\nOTSU (OpenCV):')
imshow(sudoku_otsu_auto)

## Segmentacja obrazów wielokanałowych

Segmentacja obrazów wielokanałowych (np. RGB) prostymi metodami progowania staje się problematyczne ze względu na potrzebę określenia progów w przestrzeni N-wymiarowej. Dlatego zamiast prostego progowania i segmentacji obrazów wielokanałowych częściej stosuje się metody oparte na analizie skupień.

Analiza skupień polega na znalezieniu skupisk pikseli w pewnej przestrzeni (nawet bezpośrednio w przestrzeni intensywności!) i utworzeniu w tym otoczeniu odrębnej klasy pikseli.

Dla poniższego obrazu przeprowadźmy prostą analizę intensywności pikseli.

In [ ]:
graf = cv2.imread('./graf.png', cv2.IMREAD_COLOR)
graf = cv2.resize(graf, None, fx=0.75, fy=0.75)
imshow(graf)

Obraz przedstawiony jako lista pikseli (BGR) został wyswietlony w przestrzeni 3D, gdzie koordynatami danego piksela są właśnie jego wartości intensywności. Dodatkowo piksele zostały pokolorowane zgodnie z ich intensywnościami.

In [ ]:
graf_pixels = graf.reshape([-1, 3])
pix_show(graf_pixels, 16)

Z analizy powyższej wizualizacji można wyciągnąć pare wniosków:
- większość pikseli leży na prostej pomiędzy kolorem czarnym (0, 0, 0) a białym (255, 255, 255),
- można wyróżnić pare skupisk dla kolorów:
  - czerwony,
  - niebieski,
  - złoty,
  - zielony,
  - bordowy,
  - ciemno filoletowy,

Jedną z metod podziału przestrzeni na klasy jest **Gaussian Mixture**, która polega na przybliżeniu rozkładu skupisk za pomocą N rozkładów gausowskich. Jest to metoda z parametrami uczącymi, dlatego potrzeba jest pewna próba danych, do których moglibyśmy dopasować model matematyczny.

Gotowa implementacja algorytmu znajduje się w bibliotece Scikit.

Jako dane, do których dopasujemy model wykorzystamy listę pikseli (BGR) a następnie przyporządkujemy każdemu z nich numer roskładu gausowskiego, do którego z największym prawdopodobieństwem należy.

In [ ]:
# inicjalizacja i uczenie modelu
model = GaussianMixture(n_components=8)
model.fit(graf_pixels)

# przypisanie klas do pikseli
segments = model.predict(graf_pixels)
print(segments)

Kolejnym krokiem będzie obliczenie średniego koloru dla każdej z klas (segmentów) i ponowne wyświetlenie pikseli z kolorami reprezentującymi podział na segmenty.

In [ ]:
segments_colors = np.stack([graf_pixels[segments==i].mean(0) for i in range(8)], 0)
colors = np.take(segments_colors, segments, 0)

pix_show(graf_pixels, 16, colors=colors[:, ::-1])

Piksele z przyporządkowanymi klasami to końcowa segmentacja obrazu na podstawie analizy skupień. Poniżej został przedstawiony obraz wejściowy oraz efekt segmentacji.

In [ ]:
segmented = colors.reshape(graf.shape)
imshow(np.concatenate([graf, segmented], 1))

## Segmentacja przez wykrywanie krawędzi

Segmentacja poprzez wykrywanie krawędzi sprowadza się do wiadomości poznanych na poprzednich zajęciach w ramach wykrywania punktów kluczowych, narożników i krawędzi.

Idea polega na podziale obrazu na podstawie krawędzi, a następnie wypełnieniu obszarów zamkniętych przyporządkowując kolejnym separowalnym obszarom następne identyfikatory.

Przyporządkowanie obszarom identyfikatorów zostało przedstawione w następnej sekcji.

In [ ]:
clevr = cv2.imread('./clevr.jpg', cv2.IMREAD_COLOR)
clevr = cv2.resize(clevr, None, fx=0.5, fy=0.5)
imshow(clevr)

In [ ]:
clevr_gray = cv2.cvtColor(clevr, cv2.COLOR_BGR2GRAY)
imshow(clevr_gray)

In [ ]:
canny_high, _ = cv2.threshold(clevr_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
canny_low = 0.5 * canny_high

clevr_canny = cv2.Canny(clevr_gray, canny_low, canny_high, 9)
clevr_canny = cv2.morphologyEx(clevr_canny, cv2.MORPH_CLOSE, kernel = np.ones((3, 3),np.uint8))
imshow(clevr_canny)

## Segmentacja przez rozrost

Segmentacja przez rozrost polega na iteracyjnym łączeniu przyległych ze sobą obszarów tak długo, aż nie zostanie spełniony pewien warunek. Łączenie obszarów następuje po spełnieniu **testu jednolitości**, natomiast algorytm wykonuje się tak długo aż zostanie spełniony **warunek stopu**.

**Test jednolitości** - polega na połączeniu rozważanych obszarów i sprawdzeniu pewnego warunku. Jako warunek można przyjąć np. różnicę średniej intensywności pikseli w obu obszarach. Jeśli jest większa niż pewien próg, wówczas obszary nie są jednolite i nie zachodzi połączenie.

**Warunek stopu** - warunek ten można traktować jako brak kolejnych scaleń obszarów lub jako warunek wczesnego zatrzymania algorytmu (np. gdy chcemy aby obszary nie były większe niż ustalony limit.


In [ ]:
regions = np.zeros(clevr_canny.shape[:2], np.int32)
neighbours = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def find_neighbours(img, y, x):
  c_neighbours = []
  for dy, dx in neighbours:
    ny, nx = y + dy, x + dx

    if ny < 0 or ny >= img.shape[0] or nx < 0 or nx >= img.shape[1]:
      continue

    if regions[ny, nx] > 0:
      continue

    if img[ny, nx] == 255:
      continue

    if img[y, x] == img[ny, nx]:
      c_neighbours.append((ny, nx))

  return c_neighbours


def grow_region(img, y, x, cls):
  regions[y, x] = cls

  c_neighbours = find_neighbours(img, y, x)
  for ny, nx in c_neighbours:
      regions[ny, nx] = cls

  while len(c_neighbours) > 0:
    new_neighbours = []
    for ny, nx in c_neighbours:
      i_new_neighbours = find_neighbours(img, ny, nx)
      for _ny, _nz in i_new_neighbours:
        regions[_ny, _nz] = cls

      new_neighbours.extend(i_new_neighbours)

    c_neighbours = new_neighbours


i = 1
for y in range(clevr_canny.shape[0]):
  for x in range(clevr_canny.shape[1]):
    if regions[y, x] == 0 and clevr_canny[y, x] == 0:
      grow_region(clevr_canny, y, x, i)
      i += 1

mean_colors = np.stack([np.array([255, 255, 255]) if j == 0 else clevr[regions==j].mean(0) for j in range(i)], 0)
regions_colors = np.take(mean_colors, regions, 0)

imshow(regions_colors)

# Zadania

## Zadanie 1

Na wzór sekcji o segmentacji obrazów wielokanałowych, przeprowadź taką samą analizę skupień intensywności pikseli dla obrazu './skittles.jpg' a następnie posegmentuj obraz korzystając z algorytmu K-Means (dostępnego m.in. w bibliotece scikit: sklearn.cluster.KMeans).

Przedstaw wyniki pośrednie:
- obraz wejściowy BGR
- piksele BGR w przestrzeni 3D,
- wynik segmentacji na pikselach BGR w przestrzeni 3D,
- wynik segmentacji jako obraz 2D (BGR)

In [ ]:
# obraz wejściowy BGR
skittles = cv2.imread('./skittles.jpg')
imshow(skittles)

In [ ]:
# piksele BGR w przestrzeni 3D
skittles_pixels = skittles.reshape([-1, 3])
pix_show(skittles_pixels, 16)

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(5)
model.fit(skittles_pixels)

In [ ]:
# wynik segmentacji na pikselach BGR w przestrzeni 3D
segments = model.predict(skittles_pixels)
segments_colors = np.stack([skittles_pixels[segments==i].mean(0) for i in range(5)], 0)
colors = np.take(segments_colors, segments, 0)

pix_show(skittles_pixels, 16, colors=colors[:, ::-1])

In [ ]:
# wynik segmentacji jako obraz 2D (BGR)
segmented = colors.reshape(skittles.shape)
imshow(np.concatenate([skittles, segmented], 1))

## Zadanie 2

Korzystając z metod progowania/segmentacji i znanych operacji morfologicznych, znajdź liczbę skittlesów na obrazie './skittles.jpg'.



In [ ]:
skittles = cv2.imread('./skittles.jpg')
skittles_gray = cv2.cvtColor(skittles, cv2.COLOR_BGR2GRAY)
# skittles_high, _ = cv2.threshold(skittles_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# skittles_low = 0.5 * skittles_high

skittles_canny = cv2.Canny(skittles_gray, 100, 200)
skittles_canny = cv2.dilate(skittles_canny, (1, 1), iterations=5)
# skittles_canny = cv2.erode(skittles_canny, (1, 1), iterations=1)
skittles_canny = cv2.morphologyEx(skittles_canny, cv2.MORPH_CLOSE, kernel = np.ones((3, 3),np.uint8))
imshow(skittles_canny)

contours, _ = cv2.findContours(skittles_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = list(filter(lambda x: x.max() - x.min() > 500, contours))

cv2.drawContours(skittles, contours, -1, (0, 255, 0), 2)
imshow(skittles)
print(len(contours))